In [ ]:
!pip install lxml pulsar-client

In [6]:
from bs4 import BeautifulSoup
import requests
import json
import pulsar
import logging
logging.getLogger("pulsar").setLevel(logging.WARNING)
import time
from pdf_schema import PDFInfo
from pulsar.schema import JsonSchema

In [3]:
# Testing sending messages to Pulsar--this task should be performed by the scraper
oai_uri = "http://export.arxiv.org/oai2?set=cs&verb=ListIdentifiers&metadataPrefix=oai_dc&from=2025-03-08"
page = requests.get(oai_uri)
oai_soup = BeautifulSoup(page.content, "xml")
ids = oai_soup.find_all("identifier")
urls = [id.get_text().replace('oai:arXiv.org:','https://arxiv.org/abs/') for id in ids][:10]

def process_resource(url):
    time.sleep(3)
    page = requests.get(url)
    print(url)
    soup = BeautifulSoup(page.content, "html.parser")
    title = soup.find("h1",class_='title mathjax').get_text().replace('Title:','')
    summary = soup.find("blockquote", class_='abstract mathjax').get_text().replace('Abstract:','').strip()
    authors = '\n'.join([a.get_text() + " (" + a.get('href') + ")" for a in soup.find("div",class_='authors').find_all('a')])
    pdf_url = 'https://arxiv.org' + soup.find('a',class_='download-pdf').get('href')
    return PDFInfo(title=title,summary=summary,authors=authors,url=pdf_url)
    #return json.dumps({"title": title, "summary": summary, "authors": authors, "url": pdf_url})

msgs = [process_resource(url) for url in urls]
print(len(msgs))
print(msgs[0])

https://arxiv.org/abs/1004.2931


AttributeError: 'NoneType' object has no attribute 'get_text'

In [13]:
data = json.loads('''[
  {
    "title": "Database Design - 2nd Edition",
    "authors": "Adrienne Watt",
    "description": "This second edition of Database Design book covers the concepts used in database systems and the database design process. Topics include: The history of databases Characteristics and benefits of databases Data models Data modelling Classification of database management systems Integrity rules and constraints Functional dependencies Normalization Database development process New to this edition are more examples, highlighted and defined key terms, both throughout and at the end of each chapter, and end-of-chapter review exercises. Two new chapters have been added on SQL, along with appendices that include a data model example, sample ERD exercises and SQL lab with solutions.",
    "url": "https://opentextbc.ca/dbdesign01/open/download?type=pdf",
    "category": "databases"
  },
  {
    "title": "Relational Databases and Microsoft Access 365 - Version 4.0",
    "authors": "Ron McFadyen",
    "description": "This text is a free introductory text that introduces MS Access and relational database design. The motivation is to support an introductory database system course which, to the student, is either a service course providing an introduction to database concepts, or, as a prerequisite for more advanced study in the field.",
    "url": "https://ecampusontario.pressbooks.pub/relationaldatabasesandmicrosoftaccess365/open/download?type=pdf",
    "category": "databases"
  },
  {
    "title": "The Discipline of Organizing: 4th Professional Edition",
    "authors": "Bob Glushko",
    "description": "We organize things, we organize information, we organize information about things, and we organize information about information. But even though “organizing” is a fundamental and ubiquitous challenge, when we compare these activities their contrasts are more apparent than their commonalities. We propose to unify many perspectives about organizing with the concept of an Organizing System, defined as an intentionally arranged collection of resources and the interactions they support. Every Organizing System involves a collection of resources, a choice of properties or principles used to describe and arrange resources, and ways of supporting interactions with resources. By comparing and contrasting how these activities take place in different contexts and domains, we can identify patterns of organizing. We can create a discipline of organizing in a disciplined way. The 4th edition builds a bridge between organizing and data science. It reframes descriptive statistics as organizing techniques, expands the treatment of classification to include computational methods, and incorporates many new examples of data-driven resource selection, organization, maintenance, and personalization. It introduces a new “data science” category of discipline-specific content, both in the chapter text and in endnotes, marked with [DS] in editions that contain endnotes.",
    "url": "https://berkeley.pressbooks.pub/tdo4p/open/download?type=pdf",
    "category": "databases"
  },
  {
    "title": "The Crystal Ball Instruction Manual - version 1.1 Volume One: Introduction to Data Science",
    "authors": "Stephen Davies",
    "description": "A perfect introduction to the exploding field of Data Science for the curious, first-time student. The author brings his trademark conversational tone to the important pillars of the discipline: exploratory data analysis, choices for structuring data, causality, machine learning principles, and introductory Python programming using open-source Jupyter Notebooks. This engaging read will allow any dedicated learner to build the skills necessary to contribute to the Data Science revolution, regardless of background.",
    "url": "http://stephendavies.org/crystal1.pdf",
    "category": "databases"
  },
  {
    "title": "Evidence-based Software Engineering",
    "authors": "Derek M. Jones",
    "description": "This book discusses what is currently known about software engineering, based on an analysis of all the publicly available data. This aim is not as ambitious as it sounds, because there is not a great deal of data publicly available. The intent is to provide material that is useful to professional developers working in industry; until recently researchers in software engineering have been more interested in vanity work, promoted by ego and bluster. The material is organized in two parts, the first covering software engineering and the second the statistics likely to be needed for the analysis of software engineering data.",
    "url": "http://knosof.co.uk/ESEUR/ESEUR.pdf",
    "category": "databases"
  },
  {
    "title": "Research Data Management in the Canadian Context",
    "authors": "Emily Carl and David E. Losada",  
    "description": "Research Data Management is a term for all the things that researchers do to structure, organize and maintain data before, during and after doing research. RDM is also an emerging discipline that is concerned with researching and developing ways to manage research data more effectively. But what is research data? Where is the push towards formal Research Data Management coming from? What are the requirements of good data management? Research Data Management in the Canadian Context: A Guide for Practitioners and Learners looks at these questions and more, all with a focus on Canadian guidelines, regulations and infrastructure.",
    "url": "https://ecampusontario.pressbooks.pub/canadardm/open/download?type=pdf",
    "category": "databases"
  }
]''')
msgs = [PDFInfo(title=doc['title'],summary=doc['description'],authors=doc['authors'],url=doc['url']) for doc in data]



2025-03-26 19:40:01.009 INFO  [140312862250560] ProducerStatsImpl:87 | Producer [persistent://public/default/scraper-output-test, ] , ProducerStatsImpl (numMsgsSent_ = 1, numBytesSent_ = 1706, sendMap_ = {[Key: Ok, Value: 1], }, latencyAccumulator_ = Latencies [ 50pct: 0ms, 90pct: 0ms, 99pct: 0ms, 99.9pct: 0ms], totalMsgsSent_ = 1, totalBytesSent_ = 1706, totalAcksReceived_ = , totalSendMap_ = {[Key: Ok, Value: 1], }, totalLatencyAccumulator_ = Latencies [ 50pct: 0ms, 90pct: 0ms, 99pct: 0ms, 99.9pct: 0ms])


In [16]:
procli = pulsar.Client('pulsar://pdf-service:6650')
producer = procli.create_producer(topic='scraper-output-test',schema=JsonSchema(PDFInfo))

for msg in msgs:
    producer.send(msg)

producer.close()

2025-03-26 20:02:40.027 INFO  [140313733773120] ClientConnection:193 | [<none> -> pulsar://pdf-service:6650] Create ClientConnection, timeout=10000
2025-03-26 20:02:40.027 INFO  [140313733773120] ConnectionPool:124 | Created connection for pulsar://pdf-service:6650-pulsar://pdf-service:6650-0
2025-03-26 20:02:40.029 INFO  [140312870643264] ClientConnection:410 | [10.1.1.31:46024 -> 10.102.154.227:6650] Connected to broker
2025-03-26 20:02:40.031 INFO  [140312870643264] HandlerBase:115 | [persistent://public/default/scraper-output-test, ] Getting connection from pool
2025-03-26 20:02:40.033 INFO  [140312870643264] BinaryProtoLookupService:85 | Lookup response for persistent://public/default/scraper-output-test, lookup-broker-url pulsar://localhost:6650, from [10.1.1.31:46024 -> 10.102.154.227:6650] 
2025-03-26 20:02:40.033 INFO  [140312870643264] ClientConnection:193 | [<none> -> pulsar://pdf-service:6650] Create ClientConnection, timeout=10000
2025-03-26 20:02:40.033 INFO  [14031287064

In [57]:
pulcli = pulsar.Client('pulsar://pdf-service:6650')
consumer = pulcli.subscribe(
              topic='scraper-output-test',
              subscription_name=f"test-consumer",
              schema=JsonSchema(PDFInfo) )
msg = consumer.receive()
try:
    json = msg.value()
    print(json.url)
    consumer.acknowledge(msg)
except Exception as e:
    consumer.negative_acknowledge(msg)
    print(f"Failed to process message: {e}")
finally:
    consumer.close()

2025-03-26 17:46:31.831 INFO  [140171019790144] Client:86 | Subscribing on Topic :scraper-output-test
2025-03-26 17:46:31.831 INFO  [140171019790144] ClientConnection:193 | [<none> -> pulsar://pdf-service:6650] Create ClientConnection, timeout=10000
2025-03-26 17:46:31.831 INFO  [140171019790144] ConnectionPool:124 | Created connection for pulsar://pdf-service:6650-pulsar://pdf-service:6650-0
2025-03-26 17:46:31.834 INFO  [140168925271616] ClientConnection:410 | [10.1.1.31:56734 -> 10.102.154.227:6650] Connected to broker
2025-03-26 17:46:31.838 INFO  [140168925271616] HandlerBase:115 | [persistent://public/default/scraper-output-test, test-consumer, 0] Getting connection from pool
2025-03-26 17:46:31.839 INFO  [140168925271616] BinaryProtoLookupService:85 | Lookup response for persistent://public/default/scraper-output-test, lookup-broker-url pulsar://localhost:6650, from [10.1.1.31:56734 -> 10.102.154.227:6650] 
2025-03-26 17:46:31.840 INFO  [140168925271616] ClientConnection:193 | [